# Introduction to Vector Search

Vector Search might sound scary and complicated, but it's actually really easy! And then really hard and complicated!

We computed *embeddings* of audio snippets. Now, given some example, we want to find similar audio snippets in our dataset.  To do this, we'll use vector search over the embeddings.

We'll start by doing *brute force nearest-neighbor search*. This is conceptually very easy: Just compare the query embedding to all of the target embeddings, and return the closest one(s).

We'll also find that we need to have some good data mangement along the way - it's not enough to get the best embedding, we also need to know what audio it is associated with.

In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 0 – Create a query.
# ------------------------------------------------------------
# This should mostly use code you've already written for
# previous exercises.
# 1) Create a variable for a file-path to some audio.
# 2) Load the audio.
# 3) Run the audio through an embedding model (Perch?) and get
#    the audio embedding. This is your query embedding.
# ------------------------------------------------------------


In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 1 – Create an embeddings dataset.
# ------------------------------------------------------------
# This also should rely heavily on code you've already 
# written for previous exercises... but will go a bit further.
# 1) Get all the embeddings for your dataset (eg, your 
#    favorite anuraset site, or all of the anuraset data).
# 2) Write the embeddings to disk, somehow. Check that you
#    can reload the embeddings from disk, and that the 
#    reloaded embeddings match the originals.
# 3) Given some choice of embedding from your dataset,
#    write a function which gets the audio that the embedding
#    came from.
# ------------------------------------------------------------


In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 2 – Nearest-neighbor search.
# ------------------------------------------------------------
# Now we should have a *query embedding* and a numpy array 
# of *target embeddings.*
# 1) Write a function which compares the query embedding to
#    each of the target embeddings, and finds the one that is
#    closest in Euclidean distance. Return the embedding and
#    the associated audio.
# 2) Update your function to take a `top_k` parameter. Then
#    it should return a numpy array of embeddings (with shape
#    [top_k, embedding_dim]) and an array of audio (with shape
#    [top_k, 5*sample_rate]).
# 3) Update your function to have some options for how to 
#    the nearest neighbor - try out cosine similarity and
#    maximum inner product.
# ------------------------------------------------------------


In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 3 – Displaying results.
# ------------------------------------------------------------
# Now that you have some top_k results, display them!
# Draw a spectrogram and audio player for each result.
# It's also helpful to write the filename and offset within
# the file for each result, to make it possible to go
# back and see the result in context.
#
# Extensions:
# E1) Provide a button for 'relevant' / 'irrelevant'.
# E2) Skip results which have already been marked.
# ------------------------------------------------------------


## Vector Databases in a Nutshell

Brute force search is fantastic (and fast!) until you get to millions 
or billions of embeddings.

(Micro-exercise: A 32-bit floating point number takes 4 bytes. 
How many embeddings can fit in RAM in the machine you're working on?
How many hours of audio does that number of embeddings correspond to?
How long would it take to run a brute-force search on that many embeddings?)

When the number of embeddings becomes enormous, vector databases become
sort of helpful!

The idea of a vector database is to find *approximate* nearest neighbors
quickly. This is done by *indexing* the data. There is an enormous literature
on ways to do this well, but a good cartoon-version of what works well is 
*hierarchical k-means*. You cluster the data into k clusters, then cluster
all the data assigned to each cluster centroid, and so on. Then to find 
nearest neighbors, you find the nearest top-centroid, then the nearest centroid
at the second level, and so on.

(More info than you need: this hierarchical k-means procedure has trouble when
searching for something near the cluster boundary. Various tricks can be introduced
to deal with this. There is also a family of *graph-based* indices, which
are really cool mathematically and work pretty great, but deep in the weeds.)

In my opinion, the best vector database in 2025 is called **usearch**. It's great because:

* It is self-contained, with almost no dependencies: It has one job, and it does it well.
* It can use an *on-disk* index! Most of the popular vector databases only work with embeddings in RAM, in order to give results as fast as possible. Using an on-disk index lets you scale to a much larger number of vectors for far less money.

Let's try it out!

In [ ]:
from usearch import index as uindex
import numpy as np

# For "free", here's an example of how to use usearch to index 
# and search some vectors.

# Create the index, specifying the size of the vectors, 
# data type (dtype), and the metric to use.
ui = uindex.Index(ndim=512, metric="L2sq", dtype='f16')

# Make some random data.
n = 100_000
keys = np.arange(n)
vectors = np.random.rand(n, 512).astype(np.float32)
print('\ntime to create the index: ')
%time ui.add(keys, vectors)

print(len(ui.keys))

# Make a random query.
query = np.random.rand(512).astype(np.float32)

print('\ntime to run exact search: ')
%time exact_top_k = ui.search(query, count=5, exact=True)

print('\ntime to run approximate search: ')
%time approx_top_k = ui.search(query, count=5, exact=False)

# TODO: Write some code measuring the proportion of exact_top_k
# that are in approx_top_k. This is a *recall* metric.

# Save the index to disk.
ui.save("/tmp/index.bin")

# Use the on-disk index.
ui2 = uindex.Index()
ui2.view("/tmp/index.bin")
# Check that the loaded index is the same as the original.
assert np.all(np.array(ui2.keys) == np.array(ui.keys))

print('\ntime to run approximate search from disk: ')
%time approx_disk_top_k = ui2.search(query, count=5, exact=False)


CPU times: user 2min 9s, sys: 427 ms, total: 2min 9s
Wall time: 17.9 s
100000

time to run exact search: 
CPU times: user 18.1 ms, sys: 1 μs, total: 18.1 ms
Wall time: 18 ms

time to run approximate search: 
CPU times: user 1.02 ms, sys: 0 ns, total: 1.02 ms
Wall time: 1.02 ms

time to run approximate search from disk: 
CPU times: user 3.46 ms, sys: 3.99 ms, total: 7.44 ms
Wall time: 7.44 ms


In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 4 – Embeddings usearch.
# ------------------------------------------------------------
# Do all the same stuff again, but this time with embeddings
# from the Perch model.
#
# 1) Create a usearch index for the embeddings, and insert
#    the embeddings into it.
# 2) Save the index to disk, and load it again.
# 3) Create a query embedding, and search the index for the
#    nearest neighbors.
# 4) Display the results, with a spectrogram of the query
#    and the nearest neighbors.
# ------------------------------------------------------------


In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 5 – Measuring quality.
# ------------------------------------------------------------
# Those who develop search are doomed to try to measure search
# quality. There's a lot of depth in this question, but there
# is at least a simple mechanical measure of recall which is
# useful for comparing approximate nearest neighbor search to
# exact nearest neighbor search.
#
# Search quality is typically measured by recall methods. One
# common metric is "recall@k", which is the proportion of the
# top k exact-search results that are in the top k results of 
# the approximate nearest neighbor search.
#
# 1) Write a function which takes a query embedding and a
#    usearch index, and returns the recall@k for that query.
# 2) Write a function which takes a usearch index, selects a 
#    set of N random queries, and returns the mean recall@k for 
#    those queries.
# ------------------------------------------------------------


## Limitations of ANNS

Approximate Nearest Neighbor Search does exactly one thing really well: Find the most similar stuff in a big corpus of stuff. But there are a couple key limitations to be aware of.

There are two big limitations:

1) Filtering by covariates. We often really want to ask "What's the most similar audio from this specific site / time of day / etc?"
2) Inflexibility. The ANNS search index is built against one specific metric, to get the largest results. For large datasets, the index can take a long time to create. And we can't look for 'things with a score near 0' easily.


### Filtering by Covariates

There are a few algorithms (eg, DiskANN) which have some ability to build some particular covariates into the search index, allowing some filtering for specific kinds of metadata that you identify in advance of building the index. But if you want to add more kinds of filtering later, you need to rebuild the index. And this is all bleeding-edge research anyway.

There are two somewhat reasonable workarounds:

* Do a giant ANNS search and then filter the results according to the metadata. For example, search for the top-1000, then filter/discard everything that doesn't match the parameters I care about, and return the top-5 of whatever's left. (The disadvantage is that you may get zero results back, if there's nothing in the top-1000 results matching the metadata you want.)

* Get all of the embeddings corresponding to the metadata, and then use brute-force search on the results. (The downside is that this might be slow if there's a whole lot of data matching the metadata.)

### Sampled Brute Force Search

Depending on the application, sometime sampled brute-force search is still a good idea. For margin sampling (ie, looking for classifier results with logit near 0), ANNS can't really help (you're not looking for the top-value, and you don't want to re-index to find things that are ambiguous).

Instead, you can take a random subset of embeddings and use brute-force search over that subset. Note that the recall of sampled random search is going to be approximately equal to the sample size. eg, if I take a 50% slice of the data and apply brute force search, I will on average find about 50% of the top-k results. So this can be helpful when you dont' need high recall for your ultimate goal, but instead just want something to work with quickly. Evaluating negatives or ambiguous examples is thus a good fit.

In [ ]:
# ------------------------------------------------------------
# 📝 Exercise 4 – Metadata Filtering
# ------------------------------------------------------------
# First, try some post-filtering. Write a function which
# takes the site-name (eg, 'INCT17') and a query embedding,
# runs a usearch search, and then filters the results to
# only include results from the same site.
#
# Then, try some pre-filtering. Write a function which
# takes a site-name and a query embedding. Get all of the
# embeddings from that site, and use brute-force search
# to find the nearest neighbors. 
# 
# Compare the results of the two approaches.
# ------------------------------------------------------------